https://www.jianshu.com/p/5c7b52df5e34?fbclid=IwAR1SjwZwV0vWaug7QanzVYf0gfrMWt0_cUW50vMpWBsT-KYfHAGre18Y9oc

In [128]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
train_data=pd.read_csv(r'D:\課程\大四下\資料科學應用\project2\train.csv')
test_data=pd.read_csv(r'D:\課程\大四下\資料科學應用\project2\test.csv')

In [129]:
print(train_data.isnull().sum())  

year         0
month        0
day          0
hour         0
pm2.5     1968
DEWP      5259
TEMP     10519
PRES     14025
cbwd         0
Iws          0
Is           0
Ir           0
dtype: int64


In [130]:
print(test_data.isnull().sum())  

year        0
month       0
day         0
hour        0
DEWP      876
TEMP     1752
PRES      876
cbwd        0
Iws         0
Is          0
Ir          0
dtype: int64


In [131]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    35064 non-null  int64  
 1   month   35064 non-null  int64  
 2   day     35064 non-null  int64  
 3   hour    35064 non-null  int64  
 4   pm2.5   33096 non-null  float64
 5   DEWP    29805 non-null  float64
 6   TEMP    24545 non-null  float64
 7   PRES    21039 non-null  float64
 8   cbwd    35064 non-null  object 
 9   Iws     35064 non-null  float64
 10  Is      35064 non-null  int64  
 11  Ir      35064 non-null  int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 3.2+ MB


In [132]:
train_data.corr()['pm2.5']

year    -0.017933
month    0.001527
day      0.068141
hour    -0.019552
pm2.5    1.000000
DEWP     0.199617
TEMP    -0.058037
PRES    -0.076463
Iws     -0.256205
Is       0.022720
Ir      -0.051630
Name: pm2.5, dtype: float64

In [133]:
features=['DEWP','TEMP','PRES','Iws','Is','Ir']
train_data=train_data[['pm2.5','DEWP','TEMP','PRES','Iws','Ir']]
train_y=train_data['pm2.5']
train_x=train_data.drop('pm2.5',1)
test_data=test_data[['DEWP','TEMP','PRES','Iws','Ir']]

In [134]:
'''
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(train_x)
train_x = imp.transform(train_x)
train_x=pd.DataFrame(train_x,columns=features)
train_x
test_x = imp.transform(test_data)
test_x=pd.DataFrame(test_x,columns=features)
test_x'''
train_x = train_x.fillna(method = 'ffill')
test_x = test_data.fillna(method = 'ffill')

In [136]:
print(train_x.isnull().sum()) 

DEWP    0
TEMP    0
PRES    0
Iws     0
Is      0
Ir      0
dtype: int64


In [137]:
print(test_x.isnull().sum()) 

DEWP    0
TEMP    0
PRES    0
Iws     0
Is      0
Ir      0
dtype: int64


In [149]:
train_y = train_y.fillna(method = 'ffill')
#train__y=pd.DataFrame(train_y)
print(train_y.isnull().sum()) 

24


In [151]:
train_y = train_y.fillna(method = 'bfill')
print(train_y.isnull().sum()) 

0


也要有pm2.5之變數

In [152]:
train_x['pm2.5']=train_y

In [153]:
print(train_x)

       DEWP  TEMP    PRES     Iws  Is  Ir  pm2.5
0     -21.0 -11.0  1021.0    1.79   0   0  129.0
1     -21.0 -12.0  1020.0    4.92   0   0  129.0
2     -21.0 -11.0  1020.0    6.71   0   0  129.0
3     -21.0 -11.0  1019.0    9.84   0   0  129.0
4     -20.0 -11.0  1018.0   12.97   0   0  129.0
...     ...   ...     ...     ...  ..  ..    ...
35059 -19.0   7.0  1013.0  114.87   0   0   22.0
35060 -21.0   7.0  1014.0  119.79   0   0   18.0
35061 -21.0   7.0  1014.0  125.60   0   0   23.0
35062 -21.0   7.0  1014.0  130.52   0   0   20.0
35063 -20.0   7.0  1014.0  137.67   0   0   23.0

[35064 rows x 7 columns]


min max normalize features

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(train_x)
train_x=scaler.transform(train_x)

In [60]:
time=6 #n個為一單位-->以n-1個資料預測下一個資料
x_train = [] 
y_train = []
for i in range(time,len(train_x)):
    x_train.append(train_x[i-time:i-1]) 
    y_train.append(train_y[i]) 
x_train, y_train = np.array(x_train), np.array(y_train) 


In [61]:
x_train.shape

(35053, 10, 7)

In [62]:
y_train.shape

(35053,)

In [63]:
val_num=int(len(y_train)*0.2)
x_val_new=x_train[-val_num:]
x_train_new=x_train[:-val_num]
y_val_new=y_train[-val_num:]
y_train_new=y_train[:-val_num]

In [64]:
from tensorflow.keras import callbacks

model_mckp = callbacks.ModelCheckpoint(r'D:\課程\大四下\資料科學應用\project2\project2_lstm_v5.h5',
                      monitor='val_loss',
                      save_best_only=True)

earlystop = callbacks.EarlyStopping(monitor='val_loss',
                    patience=5,
                    verbose=1)


callbacks_list = [model_mckp, earlystop]


In [65]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Flatten
from keras.optimizers import SGD

#LSTM
model = Sequential()

model.add(LSTM(100, input_shape = (x_train_new.shape[1],x_train_new.shape[2]), return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.add(Flatten())
model.add(Dense(5,activation='linear'))
model.add(Dense(1))

opt = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss="mean_absolute_error",metrics=['mean_absolute_error'], optimizer=opt)
model.fit(x_train_new, y_train_new, epochs = 20, batch_size = 100,validation_data=(x_val_new,y_val_new),callbacks=callbacks_list)

Epoch 1/20
281/281 [==============================] - 8s 27ms/step - loss: 63.5888 - mean_absolute_error: 63.5888 - val_loss: 57.8541 - val_mean_absolute_error: 57.8541
Epoch 2/20
281/281 [==============================] - 7s 24ms/step - loss: 49.7981 - mean_absolute_error: 49.7981 - val_loss: 34.8420 - val_mean_absolute_error: 34.8420
Epoch 3/20
281/281 [==============================] - 6s 22ms/step - loss: 38.7294 - mean_absolute_error: 38.7294 - val_loss: 28.2813 - val_mean_absolute_error: 28.2813
Epoch 4/20
281/281 [==============================] - 6s 22ms/step - loss: 33.5481 - mean_absolute_error: 33.5481 - val_loss: 39.0474 - val_mean_absolute_error: 39.0474
Epoch 5/20
281/281 [==============================] - 6s 22ms/step - loss: 31.2452 - mean_absolute_error: 31.2452 - val_loss: 29.6437 - val_mean_absolute_error: 29.6437
Epoch 6/20
281/281 [==============================] - 6s 22ms/step - loss: 29.3822 - mean_absolute_error: 29.3822 - val_loss: 24.6303 - val_mean_absolute_e

用前兩筆預測下一筆

In [67]:
len(train_x)

35064

In [68]:
train_x[35054:35064,:].shape

(10, 7)

In [69]:
x_data=pd.DataFrame(train_x[35054:35064],columns=['DEWP','TEMP','PRES','Iws','Is','Ir','pm2.5'])
x_data

,DEWP,TEMP,PRES,Iws,Is,Ir,pm2.5
0,0.229508,0.466667,0.459303,0.124515,0.0,0.0,0.022133
1,0.570285,0.466667,0.459303,0.141314,0.0,0.0,0.019115
2,0.245902,0.466667,0.381818,0.158113,0.0,0.0,0.016097
3,0.229508,0.450000,0.381818,0.174912,0.0,0.0,0.021127
4,0.229508,0.519980,0.400000,0.187132,0.0,0.0,0.015091
5,0.570285,0.433333,0.400000,0.195540,0.0,0.0,0.022133
6,0.196721,0.433333,0.418182,0.203948,0.0,0.0,0.018109
7,0.196721,0.433333,0.418182,0.213877,0.0,0.0,0.023139
8,0.196721,0.519980,0.418182,0.222285,0.0,0.0,0.020121
9,0.213115,0.433333,0.459303,0.234504,0.0,0.0,0.023139


In [71]:
x_data_arr=np.array(x_data)
prediction=model.predict(x_data_arr.reshape(1,10,7))

In [72]:
prediction

array([[20.893337]], dtype=float32)

In [73]:
next_data=test_x.iloc[0,:]
next_data['pm2.5']=prediction[0][0]
next_data

DEWP      -20.000000
TEMP        7.000000
PRES     1014.000000
Iws       143.480000
Is          0.000000
Ir          0.000000
pm2.5      20.893337
Name: 0, dtype: float64

In [74]:
next_data=scaler.transform(np.array(next_data).reshape(1, -1) )
next_data
#x_data.append(np.array(next_data).reshape(1,-1))

array([[0.21311475, 0.43333333, 0.41818182, 0.24443305, 0.        ,
        0.        , 0.02101945]])

In [75]:
x_data=x_data.append(pd.DataFrame(next_data,columns=['DEWP','TEMP','PRES','Iws','Is','Ir','pm2.5'])).reset_index(drop=True)
x_data=x_data.drop(0,0)
x_data

,DEWP,TEMP,PRES,Iws,Is,Ir,pm2.5
1,0.570285,0.466667,0.459303,0.141314,0.0,0.0,0.019115
2,0.245902,0.466667,0.381818,0.158113,0.0,0.0,0.016097
3,0.229508,0.450000,0.381818,0.174912,0.0,0.0,0.021127
4,0.229508,0.519980,0.400000,0.187132,0.0,0.0,0.015091
5,0.570285,0.433333,0.400000,0.195540,0.0,0.0,0.022133
6,0.196721,0.433333,0.418182,0.203948,0.0,0.0,0.018109
7,0.196721,0.433333,0.418182,0.213877,0.0,0.0,0.023139
8,0.196721,0.519980,0.418182,0.222285,0.0,0.0,0.020121
9,0.213115,0.433333,0.459303,0.234504,0.0,0.0,0.023139
10,0.213115,0.433333,0.418182,0.244433,0.0,0.0,0.021019


In [77]:
x_data_arr=np.array(x_data)
prediction=model.predict(x_data_arr.reshape(1,10,7))
prediction

array([[19.732046]], dtype=float32)

In [78]:
next_data=test_x.iloc[1,:]
next_data['pm2.5']=prediction[0][0]
next_data=scaler.transform(np.array(next_data).reshape(1, -1) )
x_data=x_data.append(pd.DataFrame(next_data,columns=['DEWP','TEMP','PRES','Iws','Is','Ir','pm2.5'])).reset_index(drop=True)
x_data=x_data.drop(0,0)

In [79]:
x_data

,DEWP,TEMP,PRES,Iws,Is,Ir,pm2.5
1,0.245902,0.466667,0.381818,0.158113,0.0,0.0,0.016097
2,0.229508,0.450000,0.381818,0.174912,0.0,0.0,0.021127
3,0.229508,0.519980,0.400000,0.187132,0.0,0.0,0.015091
4,0.570285,0.433333,0.400000,0.195540,0.0,0.0,0.022133
5,0.196721,0.433333,0.418182,0.203948,0.0,0.0,0.018109
6,0.196721,0.433333,0.418182,0.213877,0.0,0.0,0.023139
7,0.196721,0.519980,0.418182,0.222285,0.0,0.0,0.020121
8,0.213115,0.433333,0.459303,0.234504,0.0,0.0,0.023139
9,0.213115,0.433333,0.418182,0.244433,0.0,0.0,0.021019
10,0.213115,0.433333,0.400000,0.251303,0.0,0.0,0.019851


In [83]:
#formal
prediction_list=[]
x_data=pd.DataFrame(train_x[35054:35064],columns=['DEWP','TEMP','PRES','Iws','Is','Ir','pm2.5'])
x_data_arr=np.array(x_data)
prediction=model.predict(x_data_arr.reshape(1,10,7))
next_data=test_x.iloc[0,:]
next_data['pm2.5']=prediction[0][0]
prediction_list.append(prediction[0][0])
next_data=scaler.transform(np.array(next_data).reshape(1, -1) )
x_data=x_data.append(pd.DataFrame(next_data,columns=['DEWP','TEMP','PRES','Iws','Is','Ir','pm2.5'])).reset_index(drop=True)
x_data=x_data.drop(0,0)


for i in range(len(test_x)-1):
    x_data_arr=np.array(x_data)
    prediction=model.predict(x_data_arr.reshape(1,10,7))
    next_data=test_x.iloc[i+1,:]
    next_data['pm2.5']=prediction[0][0]
    prediction_list.append(prediction[0][0])
    next_data=scaler.transform(np.array(next_data).reshape(1, -1) )
    x_data=x_data.append(pd.DataFrame(next_data,columns=['DEWP','TEMP','PRES','Iws','Is','Ir','pm2.5'])).reset_index(drop=True)
    x_data=x_data.drop(0,0)
#prediction_list.append(prediction[0][0])

In [84]:
len(prediction_list)

8760

In [85]:
ID=range(1,8761)

In [86]:
ans=pd.DataFrame({'ID':ID,'Predicted':prediction_list})

In [87]:
ans

,ID,Predicted
0,1,20.893337
1,2,19.732046
2,3,19.091299
3,4,18.718378
4,5,18.595905
...,...,...
8755,8756,403.373505
8756,8757,403.153595
8757,8758,396.327911
8758,8759,384.879791


In [88]:
ans.to_csv(r'D:\課程\大四下\資料科學應用\project2\project2_lstm_v5.csv',index=False)